# <font color='violet'>Modeling</font>

This notebook builds on exploration I did with various encoding, balancing, and dimension reduction strategies in the preprocessing stage: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/7-kl-preprocess-encoding.ipynb